In [46]:
import pandas as pd
import re

In [47]:
df = pd.ExcelFile('data_eco2mix.xls').parse('Sheet1')


In [48]:
df.columns

Index(['Périmètre', 'Nature', 'Date', 'Heures', 'Consommation',
       'Prévision J-1', 'Prévision J', 'Fioul', 'Charbon', 'Gaz', 'Nucléaire',
       'Eolien', 'Solaire', 'Hydraulique', 'Pompage', 'Bioénergies',
       'Ech. physiques', 'Taux de Co2', 'Ech. comm. Angleterre',
       'Ech. comm. Espagne', 'Ech. comm. Italie', 'Ech. comm. Suisse',
       'Ech. comm. Allemagne-Belgique', 'Fioul - TAC', 'Fioul - Cogén.',
       'Fioul - Autres', 'Gaz - TAC', 'Gaz - Cogén.', 'Gaz - CCG',
       'Gaz - Autres', 'Hydraulique - Fil de l?eau + éclusée',
       'Hydraulique - Lacs', 'Hydraulique - STEP turbinage',
       'Bioénergies - Déchets', 'Bioénergies - Biomasse',
       'Bioénergies - Biogaz', ' Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'Consommation corrigée'],
      dtype='object')

In [49]:
df = df.drop(columns=["Périmètre","Nature","Consommation corrigée"], index=1)
df.drop(df.tail(1).index,inplace = True)
df.tail()

,Date,Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,Nucléaire,Eolien,...,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore
91,2024-05-11,22:45,NaN,40800.0,40600.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,2024-05-11,23:00,NaN,41700.0,41500.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,2024-05-11,23:15,NaN,41600.0,41350.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,2024-05-11,23:30,NaN,41500.0,41200.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,2024-05-11,23:45,NaN,40782.0,40500.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
cleaned_colnames = df.columns.str.normalize('NFKD')\
                                .str.encode('ascii', errors='ignore')\
                                .str.decode('utf-8')\
                                .str.strip()\
                                .str.replace(" - ","_")\
                                .str.replace(". ","_")\
                                .str.replace("?","_")\
                                .str.replace(" ","_")\
                                .str.lower()

cleaned_colnames

Index(['date', 'heures', 'consommation', 'prevision_j-1', 'prevision_j',
       'fioul', 'charbon', 'gaz', 'nucleaire', 'eolien', 'solaire',
       'hydraulique', 'pompage', 'bioenergies', 'ech_physiques', 'taux_de_co2',
       'ech_comm_angleterre', 'ech_comm_espagne', 'ech_comm_italie',
       'ech_comm_suisse', 'ech_comm_allemagne-belgique', 'fioul_tac',
       'fioul_cogen.', 'fioul_autres', 'gaz_tac', 'gaz_cogen.', 'gaz_ccg',
       'gaz_autres', 'hydraulique_fil_de_l_eau_+_eclusee', 'hydraulique_lacs',
       'hydraulique_step_turbinage', 'bioenergies_dechets',
       'bioenergies_biomasse', 'bioenergies_biogaz', 'stockage_batterie',
       'destockage_batterie', 'eolien_terrestre', 'eolien_offshore'],
      dtype='object')

In [52]:
df.columns = cleaned_colnames
df.head()

,date,heures,consommation,prevision_j-1,prevision_j,fioul,charbon,gaz,nucleaire,eolien,...,hydraulique_fil_de_l_eau_+_eclusee,hydraulique_lacs,hydraulique_step_turbinage,bioenergies_dechets,bioenergies_biomasse,bioenergies_biogaz,stockage_batterie,destockage_batterie,eolien_terrestre,eolien_offshore
0,2024-05-11,00:00,40256.0,41200.0,40800.0,120.0,0.0,496.0,35848.0,6489.0,...,7696.0,2913.0,2459.0,188.0,377.0,255.0,-14.0,5.0,5703.0,786.0
2,2024-05-11,00:30,38235.0,39400.0,38400.0,120.0,0.0,495.0,35050.0,6431.0,...,7759.0,2528.0,1419.0,190.0,378.0,255.0,0.0,25.0,5644.0,787.0
3,2024-05-11,00:45,36746.0,38150.0,37500.0,120.0,0.0,487.0,34565.0,6337.0,...,7778.0,2307.0,839.0,191.0,377.0,255.0,-42.0,4.0,5570.0,767.0
4,2024-05-11,01:00,36259.0,36900.0,36600.0,120.0,0.0,490.0,34072.0,6307.0,...,7778.0,2352.0,763.0,192.0,375.0,255.0,-7.0,5.0,5510.0,797.0
5,2024-05-11,01:15,36363.0,37000.0,36450.0,124.0,0.0,493.0,33497.0,6243.0,...,7761.0,2218.0,1349.0,183.0,375.0,255.0,-24.0,0.0,5463.0,780.0


In [53]:
df.dtypes

date                                   object
heures                                 object
consommation                          float64
prevision_j-1                         float64
prevision_j                           float64
fioul                                 float64
charbon                               float64
gaz                                   float64
nucleaire                             float64
eolien                                float64
solaire                               float64
hydraulique                           float64
pompage                               float64
bioenergies                           float64
ech_physiques                         float64
taux_de_co2                           float64
ech_comm_angleterre                   float64
ech_comm_espagne                      float64
ech_comm_italie                       float64
ech_comm_suisse                       float64
ech_comm_allemagne-belgique           float64
fioul_tac                         

In [84]:
df["date"] = pd.to_datetime(df["date"])
df['heures'] = df['heures'].astype('string')

int32_cols = ["consommation","prevision_j-1","prevision_j","nucleaire"]

int16_cols = list(df.loc[:,~df.columns.isin(["date","heures"]+int32_cols)].columns)

df[int32_cols] = df[int32_cols].apply(pd.to_numeric, 
                                      errors='coerce', downcast='integer')\
                                        .astype("Int32")

df[int16_cols] = df[int16_cols].apply(pd.to_numeric, 
                                      errors='coerce', downcast='integer')\
                                        .astype("Int16")


In [85]:
df.dtypes

date                                  datetime64[ns]
heures                                string[python]
consommation                                   Int32
prevision_j-1                                  Int32
prevision_j                                    Int32
fioul                                          Int16
charbon                                        Int16
gaz                                            Int16
nucleaire                                      Int32
eolien                                         Int16
solaire                                        Int16
hydraulique                                    Int16
pompage                                        Int16
bioenergies                                    Int16
ech_physiques                                  Int16
taux_de_co2                                    Int16
ech_comm_angleterre                            Int16
ech_comm_espagne                               Int16
ech_comm_italie                               